In [1]:
import os

import pandas as pd
import geopandas as gpd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px # type: ignore

from datetime import date, timedelta
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:
df = pd.read_parquet('Merges/PETERSON_FINAL.parquet').sort_values(by='datetime')
df.datetime = pd.to_datetime(df.datetime)

In [3]:
metric = 'chlor'
df = df[df.chlor.between(0, 15)]

app1 = Dash()

app1.layout = html.Div(children=[
    dcc.Dropdown(df.file.unique(), df.file.unique()[-1], id='dropdown-selection'),
    #dcc.Dropdown(df.columns.to_list, 'chlor', id='dropdown-selection2', style={'margin-top': '10px'}),
    dcc.Graph(id='graph-content', style={'margin-top': '10px'})
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value'),
    #Input('dropdown-selection2', 'value')
)
def update_graph(value):
    dff = df[df.file==value]
    figure = px.scatter(dff, x='datetime', y='d_from_start', color=metric, hover_data=['station_id'],
                        labels={'d_from_start': 'Distance from Start Point (km)', 'datetime': 'Datetime'},
                        title=f'{metric.capitalize()} vs time and distance from Southbay')
    figure.layout.plot_bgcolor = 'lightgrey'
    figure.layout.paper_bgcolor = 'white'
    figure.update_layout(
        title={
            'y': 0.85,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )
    return figure

if __name__ == '__main__':
    app1.run(debug=True)